In [2]:
from pytorch_metric_learning import distances, losses, miners, reducers

In [3]:
# Define the loss function
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)  # This can be customized
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)

In [ ]:
# Define the miner
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)

def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        
        # Mining triplets from embeddings
        indices_tuple = mining_func(embeddings, labels)
        
        # Calculate the loss
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss.item(), mining_func.num_triplets
                )
            )
